## Práctica N-Reinas

#### Grupo 2:
- Adrián Racero Serrano
- Juan Manuel Cardeñosa Borrego
- Raúl Casamayor Navas

## 1 - Descripción del problema y metaheurística elegida

### El Problema de las N-Reinas
El objetivo es colocar $N$ reinas en un tablero de ajedrez de $N \times N$ de tal manera que ninguna reina ataque a otra. Esto significa que no puede haber dos reinas en la misma fila, la misma columna o la misma diagonal.
Es un problema clásico de satisfacción de restricciones (CSP) con un espacio de búsqueda que crece factorialmente ($N!$), volviéndose intratable para algoritmos de fuerza bruta en valores de $N$ grandes.

### Metaheurística: Recocido Simulado (Simulated Annealing)
Para resolver este problema, hemos elegido el **Recocido Simulado**. Esta es una metaheurística de trayectoria inspirada en el proceso físico de recocido de metales.

**¿Por qué esta elección?**
* **Escape de óptimos locales:** a diferencia de una búsqueda voraz (Hill Climbing), el Recocido Simulado permite aceptar temporalmente soluciones peores con una probabilidad que depende de una "temperatura" ($T$). Esto es crucial en el problema de las N-Reinas, que tiene muchos mínimos locales.
* **Eficiencia de memoria:** solo necesita almacenar el estado actual, lo que lo hace muy ligero comparado con algoritmos poblacionales como los Genéticos.
* **Simplicidad:** su implementación es directa y permite obtener buenas soluciones en tiempos razonables para $N$ moderadamente grandes.

## 2 - Modelado del problema

Para aplicar el Recocido Simulado, necesitamos traducir el problema de las N-Reinas a una estructura matemática que el algoritmo pueda manipular.

### 2.1 - Espacio de soluciones (Representación)

Utilizaremos un **vector $S$** de dimensión $N$.

$$S = ( s_0, s_1, \dots, s_{N-1} )$$

Sujeto a las siguientes restricciones:
1. **Dominio:** $s_i \in \{0, \dots, N-1\}$ para todo $i$.
2. **Unicidad:** $\forall i, j \in \{0, \dots, N-1\} : i \neq j \implies s_i \neq s_j$

* Para cada elemento $s_i$, el **índice** ($i$) representa la **fila**.
* El **valor** $s_i$ representa la **columna** donde está la reina.
* La condición de unicidad implica que $S$ es una **permutación** de los números del $0$ al $N-1$.

**Ejemplo:** Para $N=4$, la solución `[1, 3, 0, 2]` se denota matemáticamente como $S=(1, 3, 0, 2)$:
* Fila 0 ($i=0$) $\to$ Columna 1 ($s_0=1$)
* Fila 1 ($i=1$) $\to$ Columna 3 ($s_1=3$)
* Fila 2 ($i=2$) $\to$ Columna 0 ($s_2=0$)
* Fila 3 ($i=3$) $\to$ Columna 2 ($s_3=2$)

**Ventaja:** esta representación garantiza implícitamente que **nunca habrá dos reinas en la misma columna ni en la misma fila**, reduciendo el espacio de búsqueda de $N^N$ a $N!$. El algoritmo solo tendrá que preocuparse de resolver los conflictos en las diagonales.

In [ ]:
import random
import math
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from IPython.display import display, clear_output

### Visualización de la solución

In [ ]:
def dibujar_tablero(solucion):
    """
    Dibuja un tablero tipo ajedrez y marca en rojo
    las reinas que se atacan entre sí.
    """
    N = len(solucion)

    # Detectar reinas que se atacan
    atacadas = set()
    for r1 in range(N):
        c1 = solucion[r1]
        for r2 in range(r1 + 1, N):
            c2 = solucion[r2]
            # Misma fila, columna o diagonal
            if r1 == r2 or c1 == c2 or abs(r1 - r2) == abs(c1 - c2):
                atacadas.add((r1, c1))
                atacadas.add((r2, c2))

    # Colores madera
    colores_madera = ['#f0d9b5', '#b58863']
    cmap = ListedColormap(colores_madera)

    # Tablero
    tablero = np.zeros((N, N))
    for r in range(N):
        for c in range(N):
            if (r + c) % 2 == 1:
                tablero[r, c] = 1

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(tablero, cmap=cmap, vmin=0, vmax=1, origin='upper')

    # Dibujar reinas
    for fila in range(N):
        col = solucion[fila]
        color = "red" if (fila, col) in atacadas else "black"

        plt.text(col, fila, '♛',
                 fontsize=250/N,
                 ha='center',
                 va='center',
                 color=color,
                 fontweight='bold')

    ax.set_xticks(np.arange(N))
    ax.set_yticks(np.arange(N))

    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    
    plt.tick_params(axis='both', which='both', length=0)
    plt.show()


Esta es una solución donde las 8 reinas no se atacan (solución óptima).

In [ ]:
solucion_8_reinas_optimo = [2,5,7,1,3,0,6,4]

dibujar_tablero(solucion_8_reinas_optimo)

Esta es una solución donde hay reinas que se atacan (solución subóptima).

In [ ]:
solucion_8_reinas_suboptimo = [2,1,7,5,3,0,6,4]

dibujar_tablero(solucion_8_reinas_suboptimo)

### 2.2 - Criterio de evaluación (Función de Coste)

La función objetivo, o **Energía ($E$)**, mide la calidad de una solución. En este caso, queremos minimizar el número de conflictos.

Dado que nuestra representación ya evita choques de filas y columnas, la función de coste solo necesita contar los **ataques diagonales**.
Dos reinas en las posiciones $(i, s_i)$ y $(j, s_j)$ se atacan diagonalmente si:

$$|s_i - s_j| == |i - j|$$

El objetivo del algoritmo es encontrar una configuración tal que $E(S) = 0$.

In [ ]:
def calcular_coste(solucion):
    n = len(solucion)
    ataques = 0
    for i in range(n):
        for j in range(i + 1, n):
            if abs(solucion[i] - solucion[j]) == abs(i - j):
                ataques += 1
    return ataques

In [ ]:
print('Número atques entre reinas (solución óptima) = ' + str(calcular_coste(solucion_8_reinas_optimo)))
dibujar_tablero(solucion_8_reinas_optimo)

In [ ]:
print('Número de ataques entre reinas (solución subóptima) = ' + str(calcular_coste(solucion_8_reinas_suboptimo)))
dibujar_tablero(solucion_8_reinas_suboptimo)

### 2.3 - Operador de Movimiento (Vecindad)

Para movernos por el espacio de búsqueda, definimos un operador de vecindad simple basado en el **Intercambio (Swap)**.

* **Operación:** seleccionamos aleatoriamente dos columnas distintas del tablero y e intercambiamos las filas de las reinas en esas columnas.
* **Propiedad:** al usar un swap, la nueva solución sigue siendo una permutación válida (no introduce conflictos de fila/columna), preservando la restricción fuerte del problema.

Este pequeño cambio permite al algoritmo explorar soluciones cercanas de manera eficiente.


In [ ]:
def generar_vecino(solucion):
    n = len(solucion)
    vecino = list(solucion)
    i, j = random.sample(range(n), 2)
    vecino[i], vecino[j] = vecino[j], vecino[i]
    return vecino

El siguiente código se puede ejecutar varias veces para encontrar diferentes vecinos.

In [ ]:
print('Vecino generado a partir de la solución subóptima')
vecino = generar_vecino(solucion_8_reinas_suboptimo)
print('Solución subóptima: ' + str(solucion_8_reinas_suboptimo))
print('Vecino de la solución subóptima: ' + str(vecino))
dibujar_tablero(vecino)

## 3 - Implementación del Algoritmo

A continuación, presentamos el código del **Recocido Simulado**.
El algoritmo sigue este esquema:
1.  Generar una solución inicial aleatoria.
2.  Mientras la temperatura $T$ sea alta y no hayamos encontrado la solución óptima:
    * Generar un vecino aleatorio.
    * Calcular la diferencia de energía $\Delta E$.
    * Si el vecino es mejor ($\Delta E < 0$), lo aceptamos.
    * Si es peor, lo aceptamos con una probabilidad $P = e^{-\Delta E / T}$ (Criterio de Metropolis).
    * Enfriar la temperatura ($T_{k+1} = \alpha \cdot T_k$).

In [ ]:
def generar_solucion_inicial(n):
    solucion = list(range(n))
    random.shuffle(solucion)
    return solucion

In [ ]:
def recocido_simulado(n, t_inicial, alpha, max_iter=100000):
    # 1. Escoger solución inicial
    actual_x = generar_solucion_inicial(n)
    actual_f = calcular_coste(actual_x)
    
    mejor_x = list(actual_x)
    mejor_f = actual_f
    
    t_actual = t_inicial
    iteracion = 0
    
    # Bucle unico: Hasta que se satisfaga criterio de parada
    while t_actual > 0.001 and mejor_f > 0 and iteracion < max_iter:
        
        # 2. Generar vecino
        vecino_x = generar_vecino(actual_x)
        vecino_f = calcular_coste(vecino_x)
        
        delta_f = vecino_f - actual_f
        
        aceptar = False
        
        # Criterio de aceptación (Metropolis)
        if delta_f <= 0:
            aceptar = True
        else:
            # Probabilidad p = exp(-delta / T)
            try:
                prob = math.exp(-delta_f / t_actual)
            except OverflowError:
                prob = 0
            if random.random() < prob:
                aceptar = True
        
        if aceptar:
            actual_x = vecino_x
            actual_f = vecino_f
            if actual_f < mejor_f:
                mejor_x = list(actual_x)
                mejor_f = actual_f
        
        # Actualización de Temperatura (Enfriamiento)
        t_actual = t_actual * alpha 
        iteracion += 1
        
    return mejor_x, mejor_f, iteracion

In [ ]:
N = 20
T0 = 100.0       
ALPHA = 0.999      
    
print(f"--- Recocido Simulado N={N} ---")
solucion, conflictos, _ = recocido_simulado(N, T0, ALPHA)
  
if conflictos == 0:
    print("\n✅ ¡Solución Óptima Encontrada!")
else:
    print(f"\n⚠️ Ataques restantes: {conflictos}")
    
print(f"Solución: {solucion}")

dibujar_tablero(solucion)

### 3.1 - Visualización de ejecución del algoritmo

A continuación, se ofrece una herramienta interactiva basada en `ipywidgets` que permite observar el comportamiento del algoritmo en tiempo real. Esta implementación facilita la comprensión de cómo la "temperatura" influye en la aceptación de soluciones peores y cómo el sistema converge hacia una solución estable.

**Controles disponibles:**
* **N (Reinas):** tamaño del tablero (debe ser $N \ge 4$).
* **T Inicial ($T_0$):** temperatura de partida. Valores más altos permiten más movimientos aleatorios al inicio.
* **Alpha ($\alpha$):** factor de enfriamiento ($0 < \alpha < 1$). Un valor cercano a 1 (ej. 0.99) hace que el enfriamiento sea más lento, explorando más el espacio de búsqueda.
* **Velocidad:** controla el tiempo de espera entre iteraciones para visualizar la animación cómodamente.

**Instrucciones:**
1.  Modifique los parámetros en el panel izquierdo.
2.  Pulse el botón **"Ejecutar Simulación"**.
3.  Observe en el panel derecho cómo las reinas (en rojo si están atacadas, en negro si están seguras) cambian de posición hasta encontrar una solución válida.

In [ ]:
def recocido_simulado_visualizar():
    # --- 1. DEFINICIÓN DE WIDGETS ---
    style = {'description_width': 'initial'}
    
    n_input = widgets.IntText(value=8, description='N (Reinas):', style=style)
    t0_input = widgets.FloatText(value=100.0, description='T Inicial:', style=style)
    alpha_input = widgets.FloatText(value=0.999, description='Alpha (Enfriamiento):', step=0.01, style=style)
    
    velocidad_dropdown = widgets.Dropdown(
        options=[('Lento', 0.1), ('Medio', 0.01), ('Rápido', 0.001), ('Muy Rápido', 0.0001)],
        value=0.001,
        description='Velocidad:',
        style=style
    )
    
    btn_ejecutar = widgets.Button(
        description='Ejecutar Simulación',
        button_style='success',
        icon='play'
    )
    
    out_log = widgets.Output()
    out_plot = widgets.Output()

    # --- 2. FUNCIÓN DE DIBUJADO ---
    def dibujar_estado_personalizado(n, solucion, titulo="", mostrar_ataques=True):
        # Lógica para identificar reinas atacadas solo para visualización (color rojo)
        atacadas = set()
        if mostrar_ataques:
            for r1 in range(n):
                c1 = solucion[r1]
                for r2 in range(r1 + 1, n):
                    c2 = solucion[r2]
                    if abs(r1 - r2) == abs(c1 - c2):
                        atacadas.add((r1, c1))
                        atacadas.add((r2, c2))
            
        # Colores personalizados
        colores_madera = ['#f0d9b5', '#b58863']
        cmap = ListedColormap(colores_madera)
        
        # Crear matriz del tablero
        tablero = np.zeros((n, n))
        for r in range(n):
            for c in range(n):
                if (r + c) % 2 == 1:
                    tablero[r, c] = 1
        
        fig, ax = plt.subplots(figsize=(6, 6))
        
        # Dibujar tablero
        ax.imshow(tablero, cmap=cmap, vmin=0, vmax=1, origin='upper')
        
        # Dibujar reinas
        for fila in range(n):
            col = solucion[fila]
            color = "red" if (fila, col) in atacadas else "black"
            
            # Tamaño dinámico
            fontsize = 250/n
            ax.text(col, fila, '♛', 
                    ha='center', va='center',
                    color=color, fontsize=fontsize, fontweight='bold')
        
        # --- ESTILO: EJES Y BORDES ---
        # Secuencia 0..n-1 arriba
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        ax.set_xticks(np.arange(n))
        ax.set_xticklabels(np.arange(n))
        
        # Secuencia 0..n-1 a la izquierda
        ax.set_yticks(np.arange(n))
        ax.set_yticklabels(np.arange(n))
        
        # Borde negro
        for spine in ax.spines.values():
            spine.set_edgecolor('black')
            spine.set_linewidth(1)
            
        ax.tick_params(axis='both', which='both', length=0)
        
        if titulo:
            ax.set_xlabel(titulo, fontsize=11, labelpad=10)
            
        plt.show()

    # --- 3. LÓGICA DEL ALGORITMO ---
    def ejecutar_algoritmo(b):
        out_log.clear_output()
        out_plot.clear_output()
        
        # Obtener parámetros
        n = n_input.value
        t_inicial = t0_input.value
        alpha = alpha_input.value
        delay = velocidad_dropdown.value
        
        # Chequeos de seguridad
        errores = []
        if n < 4: errores.append("❌ N debe ser mayor o igual a 4.")
        if t_inicial <= 0: errores.append("❌ T Inicial > 0.")
        if not (0 < alpha < 1): errores.append("❌ Alpha debe estar entre 0 y 1.")
            
        if errores:
            with out_log:
                for e in errores: print(e)
            return

        btn_ejecutar.disabled = True
        
        try:
            
            # 1. Escoger solución inicial
            actual_x = generar_solucion_inicial(n)
            actual_f = calcular_coste(actual_x)
            
            mejor_x = list(actual_x)
            mejor_f = actual_f
            
            t_actual = t_inicial
            iteracion = 0
            max_iter = 50000
            
            with out_log:
                print(f"🔄 Simulando N={n} | T0={t_inicial} | Alpha={alpha}...")

            # Bucle único: Hasta que se satisfaga criterio de parada
            while t_actual > 0.001 and mejor_f > 0 and iteracion < max_iter:
                
                # --- VISUALIZACIÓN ---
                saltar_frames = 10 if delay == 0.0001 else 1
                if iteracion % saltar_frames == 0:
                    with out_plot:
                        clear_output(wait=True)
                        titulo = f"Iter: {iteracion} | T: {t_actual:.2f} | Coste: {actual_f}"
                        dibujar_estado_personalizado(n, actual_x, titulo=titulo, mostrar_ataques=True)
                    time.sleep(delay)
                # ------------------------------------

                # 2. Generar vecino
                vecino_x = generar_vecino(actual_x)
                vecino_f = calcular_coste(vecino_x)
                
                delta_f = vecino_f - actual_f
                
                aceptar = False
                
                # Criterio de aceptación (Metropolis)
                if delta_f <= 0:
                    aceptar = True
                else:
                    # Probabilidad p = exp(-delta / T)
                    try:
                        prob = math.exp(-delta_f / t_actual)
                    except OverflowError:
                        prob = 0
                    if random.random() < prob:
                        aceptar = True
                
                if aceptar:
                    actual_x = vecino_x
                    actual_f = vecino_f
                    if actual_f < mejor_f:
                        mejor_x = list(actual_x)
                        mejor_f = actual_f
                
                # Actualización de Temperatura (Enfriamiento)
                t_actual = t_actual * alpha
                iteracion += 1

            # Visualización Final
            with out_plot:
                clear_output(wait=True)
                titulo_final = f"FIN: Coste = {mejor_f} en {iteracion} iteraciones"
                dibujar_estado_personalizado(n, mejor_x, titulo=titulo_final, mostrar_ataques=True)
                
            with out_log:
                if mejor_f == 0:
                    print("✅ ¡Solución Óptima Encontrada!")
                else:
                    print("⚠️ Mínimo local alcanzado (No óptimo).")
                print(f"Mejor Solución: {mejor_x}")

        except Exception as e:
            with out_log:
                print(f"❌ Error (asegúrate de haber ejecutado las celdas anteriores con las funciones auxiliares): {e}")
        finally:
            btn_ejecutar.disabled = False

    btn_ejecutar.on_click(ejecutar_algoritmo)

    # Layout
    panel_controles = widgets.VBox([
        widgets.HTML("<h3>⚙️ Configuración</h3>"),
        n_input, t0_input, alpha_input, velocidad_dropdown,
        widgets.HTML("<br>"), btn_ejecutar, widgets.HTML("<hr>"), out_log
    ])
    panel_controles.layout.width = '400px'
    panel_controles.layout.padding = '10px'
    panel_controles.layout.border = '1px solid #ddd'
    
    panel_visual = widgets.VBox([
        widgets.HTML("<h3>♛ Tablero en Tiempo Real</h3>"),
        out_plot
    ])
    panel_visual.layout.width = '600px'
    panel_visual.layout.align_items = 'center'

    display(widgets.HBox([panel_controles, panel_visual]))

# Ejecutar el widget
recocido_simulado_visualizar()

## 4 - Análisis del rendimiento del algoritmo

Evaluar el rendimiento del recocido simulado. Se realizan pruebas con diferentes tamaños de tableros (N = 8, 16, 32, ...) y se miden:
- El número de iteraciones necesarias para encontrar una solución.
- El tiempo de ejecución del algoritmo para diferentes valores de N.
- El número de soluciones subóptimas (es decir, configuraciones donde algunas reinas aún se atacan).


In [ ]:
def analizar_rendimiento(tamanos, t_inicial=100, alpha=0.999, ejecuciones_por_n=5):
    resultados = {
        'n': [],
        'tiempo_promedio': [],
        'iteraciones_promedio': [],
        'soluciones_suboptimas': []
    }
    
    print(f"{'N':<5} | {'Time Avg (s)':<12} | {'Iters Avg':<12} | {'Subóptimas':<10}")
    print("-" * 50)
    
    for n in tamanos:
        tiempos = []
        iteraciones = []
        fallos = 0
        
        for _ in range(ejecuciones_por_n):
            start_time = time.perf_counter()

            """
            alpha_ajustado = 0.995
            if n >= 20: alpha_ajustado = 0.999
            if n >= 50: alpha_ajustado = 0.9999
            """

            alpha_ajustado = alpha
            
            # Llamamos a la nueva función básica
            # Max iteraciones alto para dar tiempo a converger
            _, conflictos, iters = recocido_simulado(n, t_inicial=t_inicial, alpha=alpha_ajustado, max_iter=200000)
            
            end_time = time.perf_counter()
            
            tiempos.append(end_time - start_time)
            iteraciones.append(iters)
            
            if conflictos > 0:
                fallos += 1
        
        resultados['n'].append(n)
        resultados['tiempo_promedio'].append(np.mean(tiempos))
        resultados['iteraciones_promedio'].append(np.mean(iteraciones))
        resultados['soluciones_suboptimas'].append(fallos)
        
        print(f"{n:<5} | {np.mean(tiempos):<12.4f} | {np.mean(iteraciones):<12.1f} | {fallos:<10} de {ejecuciones_por_n}")
        
    return resultados

In [ ]:
# Tamaños a probar
tamanos_prueba = [8, 16, 32, 50, 64, 100, 128, 150] 
runs = 10 

print("Iniciando Análisis...")
datos = analizar_rendimiento(tamanos_prueba, ejecuciones_por_n=runs)

# Gráficas
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

# Tiempo
ax1.plot(datos['n'], datos['tiempo_promedio'], marker='o', color='blue')
ax1.set_title('Tiempo de Ejecución')
ax1.set_xlabel('N')
ax1.set_ylabel('Segundos')
ax1.grid(True, alpha=0.3)

# Iteraciones
ax2.plot(datos['n'], datos['iteraciones_promedio'], marker='s', color='green')
ax2.set_title('Iteraciones')
ax2.set_xlabel('N')
ax2.set_ylabel('Total Iteraciones')
ax2.grid(True, alpha=0.3)

# Fallos
bar_colors = ['red' if x > 0 else 'gray' for x in datos['soluciones_suboptimas']]
ax3.bar([str(n) for n in datos['n']], datos['soluciones_suboptimas'], color=bar_colors)
ax3.set_title(f'Subóptimas (de {runs})')
ax3.set_xlabel('N')
ax3.set_ylabel('Fallos')
ax3.set_ylim(0, runs)
ax3.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Comprobar que ocurre cuando aumentamos alpha
# (calidad de la solucion vs. tiempo de computo)
tamanos_prueba = [150] 
runs = 10 
print("Iniciando Análisis...")
datos = analizar_rendimiento(tamanos_prueba, alpha=0.9999, ejecuciones_por_n=runs)

### 4.1 - Discusión de los Resultados

Analizando las gráficas obtenidas en la ejecución anterior:

1.  **Tiempo de Ejecución (Gráfica 1):**
    * Observamos que el tiempo crece a medida que aumenta $N$, lo cual es lógico.
    * Sin embargo, el crecimiento no es "explosivo" (exponencial) como ocurriría en una búsqueda exhaustiva. Esto demuestra la **escalabilidad** del Recocido Simulado.
    * Es capaz de darnos una respuesta en cuestión de segundos incluso para tableros de $N=100$.

2.  **Número de Iteraciones (Gráfica 2):**
    * El número promedio de iteraciones necesarias para converger aumenta con $N$.
    * Esto se debe a que, en tableros más grandes, el "paisaje de energía" es más complejo y el algoritmo necesita más ciclos de enfriamiento para escapar de los óptimos locales y "relajar" el sistema hasta llegar a 0 conflictos.

3.  **Soluciones Subóptimas (Gráfica 3):**
    * Para $N$ pequeños ($N=8, 16, 32$), el algoritmo es muy robusto y casi siempre encuentra la solución óptima (0 conflictos).
    * Para $N$ muy grandes ($N > 64$), empezamos a ver barras rojas. Esto indica **fallos**: el algoritmo se "congeló" en un mínimo local antes de encontrar la solución perfecta.
    * **Conclusión:** Para resolver instancias muy grandes sin fallos, necesitaríamos un enfriamiento aún más lento (aumentar `alpha` o `max_iter`), lo que a su vez aumentaría el tiempo de ejecución. Es el clásico compromiso entre **calidad de la solución vs. tiempo de cómputo**.

## 5 - Comparación con Algoritmos Exactos

Para validar la eficiencia de nuestra metaheurística, la enfrentaremos contra un algoritmo exacto clásico: **Backtracking**.

* **Backtracking:** explora sistemáticamente el árbol de soluciones. Es completo (si hay solución, la encuentra), pero su complejidad es exponencial.
* **Recocido Simulado:** explora probabilísticamente. No garantiza encontrar la solución, pero suele ser mucho más rápido en espacios grandes.

El objetivo es encontrar el **punto de corte** ($N$) a partir del cual el enfoque exacto se vuelve inviable por tiempo, y la metaheurística se convierte en la única opción práctica.

### 5.1 - Algoritmo Exacto: Backtracking

El **Backtracking** (o Vuelta Atrás) es una técnica algorítmica para encontrar soluciones a problemas de satisfacción de restricciones de manera sistemática.

**¿Cómo funciona?**
A diferencia de las metaheurísticas que "saltan" por el espacio de soluciones, el Backtracking construye la solución paso a paso, **fila por fila**, explorando un árbol de estados implícito.

**Esquema del Algoritmo:**

1.  **Inicio:** empezamos en la fila 0 con un tablero vacío.
2.  **Paso Recursivo (`ColocarReina(fila)`):**
    * **Caso Base:** si `fila == N`, significa que hemos colocado con éxito $N$ reinas. **¡Solución encontrada!**
    * **Bucle de Búsqueda:** para cada columna `col` desde 0 hasta $N-1$:
        1.  **Validación (Poda):** verificamos si colocar una reina en `(fila, col)` es seguro (no la atacan las reinas de filas anteriores).
        2.  **Avance:** si es seguro, colocamos la reina temporalmente y llamamos recursivamente a `ColocarReina(fila + 1)`.
        3.  **Éxito:** si la llamada recursiva devuelve verdadero, propagamos el éxito hacia arriba.
        4.  **Vuelta Atrás (Backtrack):** si la llamada devuelve falso (no hay solución por ese camino), **quitamos la reina** de `(fila, col)` y probamos la siguiente columna.
3.  **Fallo:** si probamos todas las columnas y ninguna sirve, retornamos Falso (este camino no tiene solución).

**Complejidad:**
en el peor de los casos, la complejidad temporal es del orden de $O(N!)$. Esto lo hace extremadamente lento para $N > 30$, pero garantiza encontrar la solución si existe.

In [ ]:
def backtracking(n):
    """
    Algoritmo Exacto: Backtracking recursivo.
    Devuelve la primera solución válida encontrada o None.
    """
    # Usamos sets para chequeo O(1) de restricciones
    cols = set()
    diag_pos = set() # (r + c)
    diag_neg = set() # (r - c)
    
    board = [-1] * n # Índice=fila, Valor=columna

    def backtrack(fila):
        # Caso base: todas las reinas colocadas
        if fila == n:
            return True
        
        for col in range(n):
            # Verificar si la posición es segura (Poda)
            if col in cols or (fila + col) in diag_pos or (fila - col) in diag_neg:
                continue
            
            # Colocar reina
            board[fila] = col
            cols.add(col)
            diag_pos.add(fila + col)
            diag_neg.add(fila - col)
            
            # Paso recursivo
            if backtrack(fila + 1):
                return True
            
            # Backtrack (quitar reina y probar siguiente columna)
            cols.remove(col)
            diag_pos.remove(fila + col)
            diag_neg.remove(fila - col)
            board[fila] = -1
            
        return False

    if backtrack(0):
        return board
    else:
        return None

In [ ]:
N = 20

solucion = backtracking(N)

print(f"Solución: {solucion}")

dibujar_tablero(solucion)

### 5.2 - Ejecución Comparativa y Gráficas

A continuación, ejecutamos ambos algoritmos para una serie de tamaños $N$.
Dado que el Backtracking puede tardar horas para $N > 30$, hemos implementado un **límite de tiempo (TIMEOUT)** para cortar su ejecución si excede un umbral razonable, permitiendo que la gráfica muestre claramente la divergencia en rendimiento.

In [ ]:
# Tamaños a probar
# Puedes ajustar la lista si quieres probar más o menos tamaños
tamanos = [8, 12, 15, 20, 25, 30, 50, 70, 100]

# Listas para guardar los datos para la gráfica
datos_n = []
datos_bt = []
datos_sa = []

print(f"{'N':<5} | {'Backtracking (s)':<18} | {'Recocido Simulado (s)':<22} | {'Ganador':<10}")
print("-" * 65)

for n in tamanos:
    # --- 1. Medir Backtracking ---
    start_bt = time.perf_counter()
    try:
        # Límite artificial para evitar congelamientos (N > 30 es muy lento en BT)
        if n > 30:
            tiempo_bt = float('inf')
        else:
            sol_bt = backtracking(n)
            tiempo_bt = time.perf_counter() - start_bt
    except RecursionError:
        tiempo_bt = float('inf')

    # --- 2. Medir Recocido Simulado ---
    start_sa = time.perf_counter()
    sol_sa, conflictos, _ = recocido_simulado(n, t_inicial=100, alpha=0.9999)

    if conflictos > 0:
        tiempo_sa = float('inf') # No convergió a una solución válida (0 conflictos)
    else:
        tiempo_sa = time.perf_counter() - start_sa

    # --- 3. Guardar datos para la siguiente celda ---
    datos_n.append(n)
    datos_bt.append(tiempo_bt)
    datos_sa.append(tiempo_sa)

    # --- 4. Imprimir fila de la tabla ---
    str_bt = f"{tiempo_bt:.5f}" if tiempo_bt != float('inf') else "TIMEOUT"
    str_sa = f"{tiempo_sa:.5f}" if tiempo_sa != float('inf') else "NO CONVERGE"
    
    # Determinar ganador
    if tiempo_bt < tiempo_sa:
        ganador = "BT"
    elif tiempo_sa < tiempo_bt:
        ganador = "SA"
    else:
        ganador = "-" # Empate técnico o ambos fallaron
        
    print(f"{n:<5} | {str_bt:<18} | {str_sa:<22} | {ganador:<10}")

In [ ]:
# --- 5. Generar Gráfica Final ---
plt.figure(figsize=(10, 6))

# Filtrar datos para graficar (quitamos los infinitos)
x_bt = [n for n, t in zip(datos_n, datos_bt) if t != float('inf')]
y_bt = [t for t in datos_bt if t != float('inf')]

x_sa = [n for n, t in zip(datos_n, datos_sa) if t != float('inf')]
y_sa = [t for t in datos_sa if t != float('inf')]

# Dibujar líneas
plt.plot(x_bt, y_bt, marker='o', linestyle='-', color='red', label='Backtracking (Exacto)')
plt.plot(x_sa, y_sa, marker='s', linestyle='--', color='blue', label='Recocido Simulado (Metaheurística)')

# --- AJUSTE DE LÍMITES (Zoom y Margen) ---
if y_sa:
    max_sa_time = max(y_sa)
    
    # Calculamos un margen del 5% para que la línea no toque el suelo
    margin_padding = max_sa_time * 0.05
    
    # Ajustamos el límite Y:
    # - Bottom: Un poco por debajo de 0
    # - Top: Un 20% más arriba del máximo de SA (para cortar la línea roja)
    plt.ylim(-margin_padding, max_sa_time * 1.2)

# --- MODIFICACIÓN EJE X ---
# Forzamos que aparezcan exactamente los N que hemos probado
plt.xticks(datos_n)

plt.title('Comparativa de Tiempo: N-Reinas')
plt.xlabel('Número de Reinas (N)')
plt.ylabel('Tiempo de Ejecución (segundos)')
plt.legend()
plt.grid(True, alpha=0.3) # alpha hace la rejilla más suave

plt.show()

## 6 - Conclusiones

A la vista de los resultados obtenidos:

1.  **Eficiencia Temporal:** el Recocido Simulado supera drásticamente al Backtracking para $N \ge 20$. Mientras que el método exacto crece exponencialmente (timeout en $N=30$), la metaheurística mantiene tiempos de respuesta en el orden de segundos incluso para $N=100$.
2.  **Escalabilidad:** el Recocido Simulado es altamente escalable. Aunque requiere más iteraciones a medida que $N$ crece, sigue siendo viable para problemas grandes donde los métodos exactos fallan.
3.  **Compromiso Calidad-Tiempo:** hemos observado que para $N$ muy grandes, el Recocido Simulado puede no converger al óptimo (0 ataques) con los parámetros estándar. Esto evidencia el compromiso de las metaheurísticas: sacrificamos la garantía de optimalidad a cambio de poder abordar problemas complejos en tiempo razonable.